In [1]:
import os
from flask.cli import load_dotenv
import pytest
from pathlib import Path
import json
import tempfile
import shutil
from mlflow.exceptions import MlflowException
from mlflow import MlflowClient
from serve.utils.mlflow.config import MLFlowConfig, MLflowModelConfigManager
from loguru import logger

In [2]:
mlflow_docker_path = Path("../").resolve()
print(mlflow_docker_path)
env_file = mlflow_docker_path / "scripts/dev/mlflow"  / "config.env"

load_dotenv(env_file)
mlflow_port = os.getenv("MLFLOW_PORT")
minio_access_key = os.getenv("MINIO_ACCESS_KEY")
minio_secret_key = os.getenv("MINIO_SECRET_ACCESS_KEY")
mlflow_port
Path(env_file).exists()

/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/lmorbits/packages/serve


True

In [3]:
mlflow_port

'5001'

In [7]:
tracking_uri = f"http://localhost:{mlflow_port}/"
mlflow_client = MlflowClient(tracking_uri=tracking_uri, registry_uri=tracking_uri)
config_path = "./"
print(f"Config path: {config_path}")
print(f"MLflow client: {mlflow_client}")
experiments = mlflow_client.search_experiments()
print(f"Experiments: {experiments}")
model_config_manager = MLflowModelConfigManager(mlflow_client=mlflow_client, config_path=config_path)
model_dir = model_config_manager.add_model("rag_model", alias="champion" , artifact_path="model_path")

2025-03-04 23:20:31.823 | INFO     | serve.utils.model_config:__init__:66 - Initialized model configuration manager with config path: config.json
2025-03-04 23:20:31.823 | INFO     | serve.utils.mlflow.config:__init__:127 - Initialized MLflow model configuration manager


Config path: ./
MLflow client: <mlflow.tracking.client.MlflowClient object at 0x12b322a20>
Experiments: [<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1741078047476, experiment_id='1', last_update_time=1741078047476, lifecycle_stage='active', name='test_experiment', tags={}>, <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1741021252438, experiment_id='0', last_update_time=1741021252438, lifecycle_stage='active', name='Default', tags={}>]


In [8]:
from serve.experiment_tracker.mlflow.mlflow_llamacpp.manager import ModelManager
manager = ModelManager(
  mlflow_client=mlflow_client,
  config_path=Path("./c").resolve().absolute()
)
manager.add_model("qa_model", alias="champion", artifact_path="model_path")
manager.add_serve("qa_model", server_port=8000, ui_port=8080, gguf_relative_path="model_path/artifacts/model.gguf")
# manager.stop_serve_model("qa_model")
# manager.delete_model("qa_model")


2025-03-04 23:21:40.148 | INFO     | serve.servers.llamacpp.serve:__init__:54 - Initialized LLaMA CPP server manager
2025-03-04 23:21:40.149 | INFO     | serve.utils.model_config:__init__:66 - Initialized model configuration manager with config path: /Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/lmorbits/packages/serve/notebooks/c/config.json
2025-03-04 23:21:40.149 | INFO     | serve.utils.mlflow.config:__init__:127 - Initialized MLflow model configuration manager
2025-03-04 23:21:40.149 | INFO     | serve.experiment_tracker.mlflow.mlflow_llamacpp.manager:__init__:88 - Initialized model manager successfully
2025-03-04 23:21:40.160 | INFO     | serve.experiment_tracker.mlflow.mlflow_llamacpp.manager:add_model:127 - Successfully added model qa_model
2025-03-04 23:21:40.160 | DEBUG    | serve._cli.task:run:29 - ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/lmorbits/packages/serve/src/serve/servers/llamacpp'), 'serve',

In [9]:
manager.add_model("rag_model", alias="champion", artifact_path="model_path")
manager.add_serve("rag_model", server_port=8001, ui_port=8081, gguf_relative_path="model_path/artifacts/model.gguf")

2025-03-04 23:23:20.252 | WARNING  | serve.utils.model_config:load_model_config:159 - No configuration found for model: rag_model
2025-03-04 23:23:20.253 | INFO     | serve.utils.mlflow.config:check_for_update:237 - No local version found for rag_model
2025-03-04 23:23:20.253 | INFO     | serve.utils.mlflow.config:add_model:181 - Model rag_model is already up to date
2025-03-04 23:23:20.253 | WARNING  | serve.utils.model_config:load_model_config:159 - No configuration found for model: rag_model
2025-03-04 23:23:20.253 | ERROR    | serve.utils.mlflow.config:get_model_path:460 - Error getting path for rag_model: No path found for model rag_model
2025-03-04 23:23:20.263 | INFO     | serve.utils.mlflow.config:add_model:189 - Found model version: 1 for rag_model
2025-03-04 23:23:21.627 | INFO     | serve.utils.mlflow.config:download_artifacts:151 - Downloaded artifacts to /Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/lmorbits/packages/serve/notebooks/c/models/rag_model


In [11]:
manager.delete_model("qa_model")

2025-03-04 23:25:29.829 | DEBUG    | serve._cli.task:run:29 - ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/lmorbits/packages/serve/src/serve/servers/llamacpp'), 'stop', 'MODEL_ID=qa_model']
2025-03-04 23:25:40.027 | INFO     | serve._cli.task:run:35 - lmorbits-llamacpp-qa_model

2025-03-04 23:25:40.029 | INFO     | serve.servers.llamacpp.serve:stop:200 - Stopped server instance with model ID: qa_model
2025-03-04 23:25:40.030 | DEBUG    | serve._cli.task:run:29 - ['task', '--dir', PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/lmorbits/packages/serve/src/serve/servers/llamacpp'), 'delete', 'MODEL_ID=qa_model']
2025-03-04 23:25:40.088 | INFO     | serve._cli.task:run:35 - lmorbits-llamacpp-qa_model
lmorbits-llamacpp-qa_model

2025-03-04 23:25:40.089 | INFO     | serve.servers.llamacpp.serve:delete:219 - Deleted server instance with model ID: qa_model
2025-03-04 23:25:40.092 | INFO     | serve.utils.mlflow.c

In [9]:
p = Path("./c/models/model_path/artifacts/model.gguf")\
  .resolve().absolute()
p.exists()

False

In [13]:
p.parent

PosixPath('/Users/parsa/Desk/projects/university/slmops-project/slmops-thesis/lmorbits/packages/serve/notebooks/c/models/model_path/artifacts')